# B2B Ecommerce Fraud: Case Study

 Dataset comes from https://statso.io/b2b-ecommerce-fraud-case-study/

Dataset Summary:  
  
Goal: Company wants to check if **fees are charged correctly** by courier companies

Important files:

> Website Order Report 'Order Report .csv' -  order IDs, SKUs for each order  

> Master SKU 'SKU Master.csv' -  gross weight for each product (for price calculation)

> Warehouse PIN 'pincodes.csv - India Pincode mappings  

> Invoices 'Invoice.csv' - AWB Number, order ID, shipment weight, warehouse pickup PIN, customer delivery PIN, delivery area, charge per shipment, type shipment


In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import math

## Data Cleaning of E-Commerce Logistics Data (https://thecleverprogrammer.com/2025/03/02/try-these-datasets-to-master-data-cleaning/)

In [2]:
courierCompany = pd.read_csv('./Courier Company - Rates.csv')
courierCompany

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


============== SKU Master ==================

In [3]:
skuDf = pd.read_csv('./SKU Master.csv')
skuDf

,SKU,Weight (g),Unnamed: 2,Unnamed: 3,Unnamed: 4
0,8904223815682,210,NaN,NaN,NaN
1,8904223815859,165,NaN,NaN,NaN
2,8904223815866,113,NaN,NaN,NaN
3,8904223815873,65,NaN,NaN,NaN
4,8904223816214,120,NaN,NaN,NaN
...,...,...,...,...,...
61,8904223819505,210,NaN,NaN,NaN
62,8904223819499,210,NaN,NaN,NaN
63,8904223819512,210,NaN,NaN,NaN
64,8904223819543,300,NaN,NaN,NaN


In [4]:
skuDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SKU         66 non-null     object 
 1   Weight (g)  66 non-null     int64  
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.7+ KB


Drop unneccessary columns

In [5]:
skuDf = skuDf[["SKU", "Weight (g)"]]

Find duplicates

In [6]:
skuDf[skuDf['SKU'].duplicated()]

,SKU,Weight (g)
56,GIFTBOX202002,500


Drop duplicates

In [7]:
skuDf.drop_duplicates(subset=["SKU"], inplace=True)

C:\Users\mshaf\AppData\Local\Temp\ipykernel_36444\105083256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skuDf.drop_duplicates(subset=["SKU"], inplace=True)


============== Pincodes ==================

In [8]:
pincodesDf = pd.read_csv('./pincodes.csv')
pincodesDf

,Warehouse Pincode,Customer Pincode,Zone,Unnamed: 3,Unnamed: 4
0,121003,507101,d,NaN,NaN
1,121003,486886,d,NaN,NaN
2,121003,532484,d,NaN,NaN
3,121003,143001,b,NaN,NaN
4,121003,515591,d,NaN,NaN
...,...,...,...,...,...
119,121003,325207,b,NaN,NaN
120,121003,303702,b,NaN,NaN
121,121003,313301,b,NaN,NaN
122,121003,173212,e,NaN,NaN


In [9]:
pincodesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Warehouse Pincode  124 non-null    int64  
 1   Customer Pincode   124 non-null    int64  
 2   Zone               124 non-null    object 
 3   Unnamed: 3         0 non-null      float64
 4   Unnamed: 4         0 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 5.0+ KB


Drop unneccessary columns

In [10]:
pincodesDf = pincodesDf[["Warehouse Pincode", "Customer Pincode", "Zone"]]

In [11]:
pincodesDf[pincodesDf.duplicated(subset=["Warehouse Pincode", "Customer Pincode"])]

,Warehouse Pincode,Customer Pincode,Zone
16,121003,140301,b
50,121003,248001,b
72,121003,302002,b
75,121003,302017,b
77,121003,313001,b
78,121003,313001,b
82,121003,313001,b
90,121003,302017,b
101,121003,335001,b
106,121003,302031,b


In [12]:
pincodesDf.drop_duplicates(subset=["Warehouse Pincode", "Customer Pincode"], inplace=True)

C:\Users\mshaf\AppData\Local\Temp\ipykernel_36444\3913719337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pincodesDf.drop_duplicates(subset=["Warehouse Pincode", "Customer Pincode"], inplace=True)


In [13]:
pincodesDf

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
114,121003,324008,b
115,121003,302020,b
119,121003,325207,b
120,121003,303702,b


============== Order Report ==================

In [14]:
orderDf = pd.read_csv('./Order Report.csv')
orderDf

,ExternOrderNo,SKU,Order Qty,Unnamed: 3,Unnamed: 4
0,2001827036,8904223818706,1.0,NaN,NaN
1,2001827036,8904223819093,1.0,NaN,NaN
2,2001827036,8904223819109,1.0,NaN,NaN
3,2001827036,8904223818430,1.0,NaN,NaN
4,2001827036,8904223819277,1.0,NaN,NaN
...,...,...,...,...,...
395,2001806229,8904223818942,1.0,NaN,NaN
396,2001806229,8904223818850,1.0,NaN,NaN
397,2001806226,8904223818850,2.0,NaN,NaN
398,2001806210,8904223816214,1.0,NaN,NaN


In [15]:
orderDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ExternOrderNo  400 non-null    int64  
 1   SKU            400 non-null    object 
 2   Order Qty      400 non-null    float64
 3   Unnamed: 3     0 non-null      float64
 4   Unnamed: 4     0 non-null      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 15.8+ KB


Drop unnecessary columns

In [16]:
orderDf = orderDf[["ExternOrderNo", "SKU", "Order Qty"]]

In [17]:
orderDf

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


Try to check duplicates for ExternOrderNo

In [18]:
orderDf[orderDf.duplicated(subset=["ExternOrderNo"])]

,ExternOrderNo,SKU,Order Qty
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
5,2001827036,GIFTBOX202002,1.0
...,...,...,...
391,2001806233,8904223819260,1.0
393,2001806232,8904223819147,2.0
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0


It looks like there can be multiple SKUs in one external order -> Try to check duplicates for both columns

In [19]:
orderDf[orderDf.duplicated(subset=["ExternOrderNo", "SKU"])]

,ExternOrderNo,SKU,Order Qty
110,2001811363,8904223815859,1.0
281,2001807362,8904223819031,2.0
282,2001807362,8904223819024,2.0
317,2001806885,8904223819499,2.0
365,2001806471,8904223818706,1.0
366,2001806471,8904223818942,1.0
367,2001806471,8904223818850,1.0


In [20]:
orderDf[(orderDf["ExternOrderNo"] == 2001807362) & (orderDf["SKU"] == "8904223819024")]

,ExternOrderNo,SKU,Order Qty
279,2001807362,8904223819024,6.0
282,2001807362,8904223819024,2.0


Is it really a duplicate? From the previous cell we can see that the order qty varies. It can be that it billed two times?

For now, let us just add those orders up

In [21]:
orderGroupBy = orderDf.groupby(by=["ExternOrderNo", "SKU"]).sum()

In [22]:
orderGroupBy

Order Qty
ExternOrderNo SKU                     
2001806210    8904223816214        1.0
              8904223818874        1.0
2001806226    8904223818850        2.0
2001806229    8904223818706        1.0
              8904223818850        1.0
...                                ...
2001827036    8904223819093        1.0
              8904223819109        1.0
              8904223819277        1.0
              GIFTBOX202002        1.0
              SACHETS001           1.0

[393 rows x 1 columns]

In [23]:
orderGroupBy.loc[[(2001807362, '8904223819024')]]

,,Order Qty
ExternOrderNo,SKU,
2001807362,8904223819024,8.0


In [24]:
orderGroupBy = orderGroupBy.reset_index()
orderGroupBy

,ExternOrderNo,SKU,Order Qty
0,2001806210,8904223816214,1.0
1,2001806210,8904223818874,1.0
2,2001806226,8904223818850,2.0
3,2001806229,8904223818706,1.0
4,2001806229,8904223818850,1.0
...,...,...,...
388,2001827036,8904223819093,1.0
389,2001827036,8904223819109,1.0
390,2001827036,8904223819277,1.0
391,2001827036,GIFTBOX202002,1.0


In [25]:
orderGroupBy = orderGroupBy.rename(columns={"ExternOrderNo": "Order ID"})
orderGroupBy

,Order ID,SKU,Order Qty
0,2001806210,8904223816214,1.0
1,2001806210,8904223818874,1.0
2,2001806226,8904223818850,2.0
3,2001806229,8904223818706,1.0
4,2001806229,8904223818850,1.0
...,...,...,...
388,2001827036,8904223819093,1.0
389,2001827036,8904223819109,1.0
390,2001827036,8904223819277,1.0
391,2001827036,GIFTBOX202002,1.0


============== Invoice ==================

In [26]:
invoiceDf = pd.read_csv('./Invoice.csv')
invoiceDf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [27]:
invoiceDf[invoiceDf.duplicated(subset=["Order ID"])]

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)


## Fraud Analysis

Steps plan:  

1. Calculate the weight for each SKU in each order. SKU weight per unit x quantity order  
2. Group by order and get the total weight per order  
3. Round to the nearest multiple of 0.5kg -> Conversion to kg needed  
4. Get receiver delivery zone as per ABC (the company) while get the respective fixed and additional  
5. Calculate the expected price and investigate if the company is being over-/undercharged or charged properly  

Step 1: Calculate the weight for each SKU in each order. SKU weight per unit x quantity order

In [35]:
result = orderGroupBy.copy()
result

,Order ID,SKU,Order Qty
0,2001806210,8904223816214,1.0
1,2001806210,8904223818874,1.0
2,2001806226,8904223818850,2.0
3,2001806229,8904223818706,1.0
4,2001806229,8904223818850,1.0
...,...,...,...
388,2001827036,8904223819093,1.0
389,2001827036,8904223819109,1.0
390,2001827036,8904223819277,1.0
391,2001827036,GIFTBOX202002,1.0


In [36]:
result = result.merge(skuDf, on="SKU", how="left")
result["Total Weight (g)"] = result["Order Qty"] * result["Weight (g)"]

Step 2. Group by order and get the total weight per order

In [37]:
resultOrder = result.groupby(by="Order ID").sum()
resultOrder = resultOrder[["Total Weight (g)"]]
resultOrder

,Total Weight (g)
Order ID,
2001806210,220.0
2001806226,480.0
2001806229,500.0
2001806232,1302.0
2001806233,245.0
...,...
2001821995,477.0
2001822466,1376.0
2001823564,672.0


Step 3: Round to the nearest multiple of 0.5kg -> Conversion to kg needed

In [38]:
resultOrder = resultOrder.apply(lambda x: x/1000)
resultOrder = resultOrder.rename(columns={'Total Weight (g)': 'Total Weight (kg)'})
resultOrder

,Total Weight (kg)
Order ID,
2001806210,0.220
2001806226,0.480
2001806229,0.500
2001806232,1.302
2001806233,0.245
...,...
2001821995,0.477
2001822466,1.376
2001823564,0.672


In [39]:
resultOrder['Weight Slab (kg)'] = resultOrder['Total Weight (kg)'].apply(lambda x: math.ceil(x / 0.5) * 0.5)
resultOrder

,Total Weight (kg),Weight Slab (kg)
Order ID,,
2001806210,0.220,0.5
2001806226,0.480,0.5
2001806229,0.500,0.5
2001806232,1.302,1.5
2001806233,0.245,0.5
...,...,...
2001821995,0.477,0.5
2001822466,1.376,1.5
2001823564,0.672,1.0


In [40]:
pincodesDf

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
114,121003,324008,b
115,121003,302020,b
119,121003,325207,b
120,121003,303702,b


In [45]:
resultOrder.reset_index()

,Order ID,Total Weight (kg),Weight Slab (kg)
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


Step 4: Get receiver delivery zone as per ABC (the company) while get the respective fixed and additional prices

In [64]:
finalDf = pd.merge(resultOrder, invoiceDf, on="Order ID")
finalDf

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


In [65]:
finalDf['Type of Shipment'].unique()

array(['Forward charges', 'Forward and RTO charges'], dtype=object)

In [66]:
finalDf['Type of Shipment'] = finalDf['Type of Shipment'].apply(lambda x : "fwd" if x == "Forward charges" else "rto")
finalDf
# finalDf['Type of Shipment'] = "fwd" if finalDf['Type of Shipment'] == "Forward charges" else "rto"

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,fwd,174.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,fwd,90.2
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,fwd,90.2
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,fwd,135.0
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,fwd,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,fwd,45.4
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,fwd,135.0
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,rto,172.8
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,rto,345.0


In [ ]:
finalDf["stringPrefix"] = finalDf['Type of Shipment'] + "_" + finalDf["Zone"]

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),stringPrefix
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,fwd,174.5,fwd_b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,fwd,90.2,fwd_d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,fwd,90.2,fwd_d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,fwd,135.0,fwd_d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,fwd,61.3,fwd_b
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,fwd,45.4,fwd_d
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,fwd,135.0,fwd_d
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,rto,172.8,rto_d
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,rto,345.0,rto_d


In [72]:
finalDf['FixedPrice'] = finalDf['stringPrefix'].apply(lambda x: courierCompany[f"{x}_fixed"][0])
finalDf['AdditionalPrice'] = finalDf['stringPrefix'].apply(lambda x: courierCompany[f"{x}_additional"][0])

In [73]:
finalDf

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),stringPrefix,FixedPrice,AdditionalPrice
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,fwd,174.5,fwd_b,33.0,28.3
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,fwd,90.2,fwd_d,45.4,44.8
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,fwd,90.2,fwd_d,45.4,44.8
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,fwd,135.0,fwd_d,45.4,44.8
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,fwd,61.3,fwd_b,33.0,28.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,fwd,45.4,fwd_d,45.4,44.8
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,fwd,135.0,fwd_d,45.4,44.8
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,rto,172.8,rto_d,41.3,44.8
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,rto,345.0,rto_d,41.3,44.8


Step 5: Calculate the expected price and investigate if the company is being over-/undercharged or charged properly

In [84]:
finalDf['ExpectedPrice'] = (finalDf['FixedPrice'] + (finalDf['Weight Slab (kg)'] / 0.5 * finalDf['AdditionalPrice'])).round(2)
finalDf

,Order ID,Total Weight (kg),Weight Slab (kg),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),stringPrefix,FixedPrice,AdditionalPrice,ExpectedPrice
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,fwd,174.5,fwd_b,33.0,28.3,61.3
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,fwd,90.2,fwd_d,45.4,44.8,90.2
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,fwd,90.2,fwd_d,45.4,44.8,90.2
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,fwd,135.0,fwd_d,45.4,44.8,179.8
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,fwd,61.3,fwd_b,33.0,28.3,61.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,fwd,45.4,fwd_d,45.4,44.8,90.2
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,fwd,135.0,fwd_d,45.4,44.8,179.8
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,rto,172.8,rto_d,41.3,44.8,130.9
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,rto,345.0,rto_d,41.3,44.8,220.5


Charged honestly

In [88]:
dfEqual = finalDf[finalDf['Billing Amount (Rs.)'] == finalDf['ExpectedPrice']]

Undercharge

In [92]:
dfLess = finalDf[finalDf['Billing Amount (Rs.)'] < finalDf['ExpectedPrice']]

In [99]:
differeceValueLess = (dfLess['ExpectedPrice'].sum() - dfLess['Billing Amount (Rs.)'].sum()).round(2)

OverCharge

In [96]:
dfMore = finalDf[finalDf['Billing Amount (Rs.)'] > finalDf['ExpectedPrice']]

In [98]:
differeceValueMore = (dfMore['ExpectedPrice'].sum() - dfMore['Billing Amount (Rs.)'].sum()).round(2)

In [102]:
dfResult = pd.DataFrame({
    'Description': ['Total Orders where ABC has been correctly charged', 'Total Orders where ABC has been overcharged', 'Total Orders where ABC has been undercharged'],
    'Count': [len(dfEqual), len(dfMore), len(dfLess)],
    'Difference Amount (Rs.)': [0, differeceValueMore, differeceValueLess]
})

In [103]:
dfResult

,Description,Count,Difference Amount (Rs.)
0,Total Orders where ABC has been correctly charged,41,0.0
1,Total Orders where ABC has been overcharged,24,-1613.4
2,Total Orders where ABC has been undercharged,59,2720.5
